In [ ]:
import requests
from bs4 import BeautifulSoup as bs

r = requests.get('https://cryptoslam.io/')
content = bs(r.content)
table = content.select('table.table-hover.js-top-by-sales-table-all.summary-sales-table')[0]
script = table.find_all('a', href=True)
lista = []
for s in script:
    if s["href"][0] == "/":
        lista.append(s["href"])
lista2 = []
for i in lista:
    stringa = i.replace("/sales/summary", "")
    stringa = stringa.replace("/sales", "")
    stringa = stringa.replace("/", "")
    if stringa not in lista2:
        lista2.append(stringa)

lista2 = lista2[:250]
print(lista2)

In [ ]:
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import time

ts = int(time.time()*1000)
endTime = (datetime.today()+timedelta(days=1)).strftime("%Y-%m-%d")
eth_df = yf.download('ETH-USD',
                      start = '2019-01-01',
                      end = endTime,
                      progress = False)
eth = eth_df.reset_index()
eth = eth.loc[:,['Date', 'Close']]
eth.columns = ['Date', 'ETH']
date_ETH = list(eth.Date)
lista_date_ETH = []
for stringa in date_ETH:
    timeStr = stringa.strftime("%Y-%m-%d")
    lista_date_ETH.append(timeStr)
val_ETH = list(eth.ETH)
c = dict(zip(lista_date_ETH, val_ETH))

In [ ]:
import re
import pandas as pd

for fil in range(0,250):
    try:
        URL = "https://cryptoslam.io/"+lista2[fil]+"/stats/AllTime?_="+str(ts)
        r = requests.get(URL)
        soup = bs(r.content, 'html.parser')
        script = soup.find_all('p', {"class":"statistics-item__title-header"})
        lista_titoli = []
        lista_valori = []
        for i in script:
            titolo = str(i)
            titolo = re.sub(r'[^A-Za-z0-9\s]', " ", titolo)
            titolo = titolo.replace("p class  statistics item  title header  ", "")
            titolo = titolo.replace("  p", "")
            lista_titoli.append("Data")
            lista_titoli.append(titolo)
            lista_valori.append(titolo)
        lista_titoli.insert(2, "Data")
        lista_titoli.insert(3, "ETH")
        lista_valori.insert(1, "ETH")

        ###### SALES DATE TO UNIFY DATASET

        for i in range(0,1):
            data_Sales = str(lista[2])
            data_Sales = data_Sales.split("\"")
            data_Sales = str(data_Sales)
            chars = ["\\", ":", ","]
            for char in chars:
                data_Sales = data_Sales.replace(char, "")
            data_Sales = data_Sales.replace("-", "_")
            data_Sales = re.sub(r'[^\w]', " ", data_Sales)
            data_Sales = str(data_Sales)
            data_Sales = data_Sales.replace("_", "-")
            data_Sales = data_Sales.split(" ")
            data_Sales = list(filter(None, data_Sales))
            data_Sales = data_Sales[6:]
            for j in data_Sales[1::2]:
                data_Sales.remove(j)
            data_unic = []
            for j in data_Sales:
                j=j[:-8]
                data_unic.append(j)

        ###### BUILDING DATASET

        ### VALUES
        lista_stats = []
        for i in range((len(lista_titoli)-2)//2):
            script2 = soup.find_all('script')[i].text.strip()[50:-1]
            lista = script2.split(";")
            stringa = str(lista[0])
            stringa = stringa.replace("]')", "")
            lista_final = stringa.split(",")

        ### DATES
            date = str(lista[2])
            date = date.split("\"")
            date = str(date)
            chars = ["\\", ":", ","]
            for char in chars:
                date = date.replace(char, "")
            date = date.replace("-", "_")
            date = re.sub(r'[^\w]', " ", date)
            date = str(date)
            date = date.replace("_", "-")
            date = date.split(" ")
            date_final = list(filter(None, date))
            date_final = date_final[6:]
            for j in date_final[1::2]:
                date_final.remove(j)
            lista_date = []
            for j in date_final:
                j=j[:-8]
                lista_date.append(j)
            stats_dataset = []
            date_dataset = []
            d = dict(zip(lista_date, lista_final))
            for k,v in d.items():
                if k in data_unic:
                    stats_dataset.append(v)
                    date_dataset.append(k)
            lista_stats.append(date_dataset)
            lista_stats.append(stats_dataset)
            stats_ETH = []
            date_ETH = []
            if i == 0:
                for k,v in c.items():
                    if k in data_unic:
                        stats_ETH.append(v)
                        date_ETH.append(k)
                lista_stats.append(date_ETH)
                lista_stats.append(stats_ETH)
            else:
                continue

        lista_nft = []
        lista_tit = []
        for i in range(0, len(lista_stats)):
            if len(lista_stats[i]) >= len(data_unic)*2//5:
                lista_nft.append(lista_stats[i])
                lista_tit.append(lista_titoli[i])
        lista_tit_nft = lista_tit[1::2]
        df = pd.DataFrame(lista_nft)
        df0 = df.transpose()
        df1 = df0.set_index(0)
        df1 = df1.reset_index()
        zipped = zip(df1.columns[::2], df1.columns[1::2])
        df1 = pd.concat([df1.groupby(a)[b].mean() for a, b in zipped], axis=1)
        df1 = df1.transpose()
        df1.insert(0,"Date", lista_tit_nft, True)
        df1 = df1.transpose()
        df1.to_excel (r'NFT DIRECTORY'+lista2[fil]+".xlsx", header=False)
    except:
        continue